In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
#from sklearn.preprocessing import StandardScaler


%store -r X_features_full
%store -r X_features_poly_full
%store -r y_labels_full

%store -r totaldf_full
totaldf = totaldf_full
X_features = X_features_full
X_features_poly = X_features_poly_full
y_labels = y_labels_full

In [2]:
voting_reg = joblib.load("voting_reg.pkl")
poly_ridge_reg = joblib.load("poly_ridge_reg.pkl")
svm_rbf_reg = joblib.load("svm_rbf_reg.pkl")
forest_reg = joblib.load("forest_regressor.pkl")
seq_model = keras.models.load_model('D:\python\\full data\\first')



In [3]:
totaldf.insert(7,'poly ridge μ', totaldf.μob*(np.exp(np.power(poly_ridge_reg.predict(X_features_poly[2]),2)) -0.0012))
totaldf.insert(8,'rbf μ', totaldf.μob*(np.exp(np.power(svm_rbf_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(9,'forest μ', totaldf.μob*(np.exp(np.power(forest_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(10,'voting μ', totaldf.μob*(np.exp(np.power(voting_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(11,'sequential', (np.exp(np.power(seq_model.predict(X_features[2]),2)) -0.0012))
totaldf.sequential = totaldf.sequential*totaldf.μob
totaldf.insert(12,'Almehaideb', totaldf.μob*np.power(totaldf.P/totaldf.Pb,0.134819+1.94345*np.power(10,-4.0)*totaldf.Rsob-1.93106*np.power(10,-9.0)*np.power(totaldf.Rsob,2)))
X = 0.776644115+0.987658646*np.log10(totaldf.μob)-0.190564677*np.log10(totaldf.Rsob)+0.009147711*totaldf.μob*np.log10(totaldf.Rsob)-0.000019111*(totaldf.P-totaldf.Pb)

totaldf.insert(13,'Dindoruk and Christman',totaldf.μob +0.00006334*totaldf.ΔP*np.power(10,X))

72/72 [==============================] - 0s 1ms/step


In [4]:
def Stats_matrix(a):
    average_error = []
    standard_deviation = []
    absolute_average_error = []
    absolute_standard_deviation =[]
    for i in range(7):
        average_error.append(np.average((a.iloc[:,i+7]-a.μ)/a.μ*100))
        standard_deviation.append(np.std((a.iloc[:,i+7]-a.μ)/a.μ*100))
        absolute_average_error.append(np.average(np.absolute((a.iloc[:,i+7]-a.μ)/a.μ*100)))
        absolute_standard_deviation.append(np.std(np.absolute((a.iloc[:,i+7]-a.μ)/totaldf.μ*100)))
    statistics  = pd.DataFrame(list(zip(average_error,standard_deviation,absolute_average_error,
                                        absolute_standard_deviation)),columns = ['% AE','% Std Dev AE','%AAE','% Std Dev AAE'],index = ['Ridge Regression','SVM RBF kernel','Random Forest','Voting Regressor','ANN with 1 layer','Beal','Kouzel'])
    return statistics

Stats_matrix(totaldf)

,% AE,% Std Dev AE,%AAE,% Std Dev AAE
Ridge Regression,-0.061266,4.654700,2.921706,3.624033
SVM RBF kernel,-0.530577,4.454980,2.750931,3.544113
Random Forest,0.033931,4.617648,2.792770,3.677535
Voting Regressor,-0.288119,4.392745,2.680833,3.491756
ANN with 1 layer,-0.049976,4.628986,2.703566,3.757757
Beal,2.868533,10.281060,6.464934,8.493134
Kouzel,6.067123,15.541939,7.501041,14.902894
